##  Cross-Validation and the Bootstrap

In [88]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,summarize,poly)
from sklearn.model_selection import train_test_split

In [89]:
from functools import partial
from sklearn.model_selection import (cross_validate,KFold,ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

#### The Validation Set Approach

In [90]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,test_size=196,random_state=0)

In [91]:
Auto_train.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
249,20.2,8,302.0,139,3570,12.8,78,1,mercury monarch ghia
24,21.0,6,199.0,90,2648,15.0,70,1,amc gremlin
261,17.7,6,231.0,165,3445,13.4,78,1,buick regal sport coupe (turbo)
44,18.0,6,258.0,110,2962,13.5,71,1,amc hornet sportabout (sw)
225,19.0,6,225.0,100,3630,17.7,77,1,plymouth volare custom


In [107]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train,X_train)
results = model.fit()
results

In [108]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [147]:
def evalMSE(terms,response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    
    model = sm.OLS(y_train,X_train) 
    results = model.fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [151]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],'mpg',Auto_train,Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

##### If we choose different different split of training and validation set

In [153]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
    
MSE

array([20.75540796, 16.94510676, 16.97437833])

In [163]:
hp_model = sklearn_sm(sm.OLS,MS(['horsepower']))

X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,X,Y,cv=Auto.shape[0])

cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929216

In [191]:
#print(cv_results.keys())

In [167]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,X,Y,cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323073])

##### Cross-validation using K-fold 

In [197]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0) #use same splits for each degree

for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,X,Y, cv=cv)
    
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848399, 19.13721595])

##### Another spliting mechanism using ShuffleSplit() function to perform the validation set approach

In [199]:
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)

results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv = validation)

results['test_score']

array([23.61661707])

#### Estimate the variability in the test error

In [203]:
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)

results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv = validation)

results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.421845094109181)

#### The Bootstrap

In [208]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [209]:
alpha_func(Portfolio, range(100))

0.57583207459283

In [216]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619002

In [211]:
 np.random.default_rng(0)

Generator(PCG64) at 0x240366F83C0

In [217]:
def boot_SE(func,D,n=None,B=1000,seed=0):
    
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    
    for _ in range(B):
        idx = rng.choice(D.index,n,replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [218]:
alpha_SE = boot_SE(alpha_func,Portfolio,B=1000,seed=0)
alpha_SE

0.09118176521277638

#### Estimating the Accuracy of a Linear Regression Model

In [115]:
def evalMSE(terms,response,train,test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)


In [195]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.outer(A, B)

array([[ 6, 12],
       [10, 20],
       [18, 36]])